## <center> Image recognition </center>
### <center> Train your own model vs Transfer learning </center>

In [23]:
import numpy as np
from keras.datasets import cifar10
from keras.models import Sequential, model_from_json
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from keras.utils import to_categorical, load_img
from keras.callbacks import TensorBoard
from pathlib import Path

### I. Loading data

In [24]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [25]:
print('X_train shape: {}'.format(X_train.shape))
print('X_test shape: {}'.format(X_test.shape))

X_train shape: (50000, 32, 32, 3)
X_test shape: (10000, 32, 32, 3)


- CIFAR10 is 60K image split into 50K on train and 10 on test. Each image size is: 32 x 32 x 3

### II. Preprocessing images

#### 1. Normalisation

First, let's convert our data into float variables

In [26]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

Next, let's divide by 255 to normalize data between 0 & 1

In [27]:
X_train /= 255
X_test /= 255

#### 2. Convert labels to categorical

In [28]:
y_train = to_categorical(y_train, num_classes = 10)
y_test = to_categorical(y_test, num_classes = 10)

### III. Trained from scratch model

#### 1. Model creation

In [30]:
model = Sequential()

# First convolution block
model.add(Conv2D(32, (3, 3), padding="same", input_shape=(32, 32, 3), activation="relu"))
model.add(Conv2D(32, (3, 3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Second convolution block
model.add(Conv2D(32, (3, 3), padding="same", activation="relu"))
model.add(Conv2D(32, (3, 3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Third convolution block
model.add(Conv2D(32, (3, 3), padding="same", activation="relu"))
model.add(Conv2D(32, (3, 3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Flatten then dense layer
model.add(Flatten())
model.add(Dense(256, activation="relu"))
model.add(Dropout(0.3))
# Output layer
model.add(Dense(10, activation="softmax"))

# Model compilation
model.compile(loss = 'categorical_crossentropy',
              optimizer = 'adam',
              metrics = ['accuracy'])

# Tensorboard logging
logger = TensorBoard(log_dir='logs/3Conv_blocks', write_graph=True, histogram_freq=3)

In [31]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 32, 32, 32)        896       
                                                                 
 conv2d_13 (Conv2D)          (None, 32, 32, 32)        9248      
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 16, 16, 32)       0         
 2D)                                                             
                                                                 
 dropout_8 (Dropout)         (None, 16, 16, 32)        0         
                                                                 
 conv2d_14 (Conv2D)          (None, 16, 16, 32)        9248      
                                                                 
 conv2d_15 (Conv2D)          (None, 16, 16, 32)        9248      
                                                      

#### 2. Training the model

In [32]:
model.fit(
    X_train,
    y_train,
    shuffle=True,
    batch_size=32,
    epochs=30,
    validation_data=(X_test, y_test),
    callbacks=[logger]
)

Epoch 1/30
1563/1563 [==============================] - 41s 24ms/step - loss: 1.6862 - accuracy: 0.3730 - val_loss: 1.3368 - val_accuracy: 0.5086
Epoch 2/30
1563/1563 [==============================] - 40s 26ms/step - loss: 1.3113 - accuracy: 0.5258 - val_loss: 1.1875 - val_accuracy: 0.5671
Epoch 3/30
1563/1563 [==============================] - 42s 27ms/step - loss: 1.1673 - accuracy: 0.5820 - val_loss: 1.0401 - val_accuracy: 0.6371
Epoch 4/30
1563/1563 [==============================] - 41s 26ms/step - loss: 1.0709 - accuracy: 0.6189 - val_loss: 0.9487 - val_accuracy: 0.6663
Epoch 5/30
1563/1563 [==============================] - 45s 29ms/step - loss: 1.0093 - accuracy: 0.6409 - val_loss: 0.8926 - val_accuracy: 0.6870
Epoch 6/30
1563/1563 [==============================] - 49s 31ms/step - loss: 0.9640 - accuracy: 0.6590 - val_loss: 0.8352 - val_accuracy: 0.7073
Epoch 7/30
1563/1563 [==============================] - 44s 28ms/step - loss: 0.9208 - accuracy: 0.6737 - val_loss: 0.8316 -

#### 3. Saving the model's structure

In [33]:
model_structure = model.to_json()
f = Path('model_structure.json')
f.write_text(model_structure)

5872

#### 4. Saving the model's weights

In [34]:
model.save_weights('model_weights.h5')

#### 5. Loading the model

- Loading the model's structure

In [35]:
p = Path('model_structure.json')
model_structure = p.read_text()

In [36]:
model = model_from_json(model_structure)

- Loading the model parameters

In [37]:
model.load_weights('model_weights.h5')
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

#### 6. Model evaluation

In [38]:
model.evaluate(X_test, y_test)

313/313 [==============================] - 3s 9ms/step - loss: 0.6620 - accuracy: 0.7720


[0.662005603313446, 0.7720000147819519]

#### 7. Model prediction

- Loading external images to test

In [39]:
frog = load_img("test_images/frog.png", target_size=(32, 32))
bay = load_img("test_images/bay.jpg", target_size=(32, 32))
dog = load_img("test_images/dog.png", target_size=(32, 32))

In [40]:
class_labels = [
    "Plane",
    "Car",
    "Bird",
    "Cat",
    "Deer",
    "Dog",
    "Frog",
    "Horse",
    "Boat",
    "Truck"
]

- Predicting the 3 test image's class

In [41]:
pred = model.predict(np.stack([frog, bay, dog], axis=0))
pred_classes = []
for prob in pred:
    most_likely = int(np.argmax(prob))
    class_label = class_labels[most_likely]
    pred_classes.append(class_label)
print(pred_classes)

1/1 [==============================] - 0s 234ms/step
['Frog', 'Car', 'Car']
